# Exercise 1

Lets imagine there is a medical test for a disease that has a 92% chance of detecting a disease if the person has it (True Positive) and a 94% chance of correctly detecting that the disease is absent if the person doesnt have the disease (True Negative).

10% of the general population has the disease.

### What is the probability of testing positive (whether you have the disease or not)?

Law of Total Probability

P(+) = P(+|has)P(has) + P(+|not)P(not) = (0.92)(0.1) + (0.06)(0.9) = 0.146

### What is the probability of having the disease given that you got a positive test?

Bayes' Theorem

P(has|+) = P(+|has)P(has) / P(+) = (0.92)(0.1) / 0.146 = 0.63

# Exercise 2

We are going to use a reduced sample of the [Lending Club dataset](https://www.lendingclub.com/info/download-data.action). The Lending Club is a startup that provides personal loans up to $40,000. 

Every quarter they release all of their data related to how many loan requests they accept and how many they reject.

The target variable is `status`, that indicates whether the loan request was `accepted` or `rejected`.

In [20]:
import pandas as pd

loans = pd.read_csv("data/loans.csv")

target = "status"

Don't need zip code because state should be fine as a location variable; and there are so many zip codes in the country

In [46]:
loans = loans.drop(columns=['zip_code'])

KeyError: "labels ['zip_code'] not contained in axis"

In [47]:
loans['status'] = loans['status'].map({'accepted': 1, 'declined': 0})

In [52]:
loans.tail()

,addr_state,debt_to_income_ratio,emp_length,loan_amnt,status,title
9995,CA,0.0863,< 1 year,5000.0,0,Debt consolidation
9996,OR,2.3776,< 1 year,1000.0,0,Debt consolidation
9997,WI,0.0057,< 1 year,1500.0,0,Debt consolidation
9998,CO,0.0000,< 1 year,1000.0,0,Debt consolidation
9999,NC,1.0000,< 1 year,10000.0,0,Debt consolidation


In [3]:
loans.dtypes

addr_state               object
debt_to_income_ratio     object
emp_length               object
loan_amnt               float64
status                   object
title                    object
zip_code                 object
dtype: object

In [32]:
loans.emp_length.unique()

array(['8 years', '< 1 year', '3 years', '10+ years', '1 year', '9 years',
       '5 years', '4 years', '2 years', '6 years', '7 years'],
      dtype=object)

In [55]:
loans.isnull().values.any()

False

In [54]:
loans.emp_length = loans.emp_length.fillna('< 1 year')
loans.debt_to_income_ratio = loans.debt_to_income_ratio.fillna(loans.debt_to_income_ratio.mean())

In [6]:
type(loans.debt_to_income_ratio.unique()[0])

str

Turned all values in debt_to_income_ratio into floats

In [22]:
debt_to_income  = []
for ratio in loans.debt_to_income_ratio:
    if type(ratio) == float:
        debt_to_income.append(float("{0:.4f}".format(ratio)))
    elif type(ratio) == str:
        if ratio[-1] == '%':
            debt_to_income.append(float("{0:.4f}".format(float(ratio[:-1]) / 100)))
        else:
            debt_to_income.append(float("{0:.4f}".format(float(ratio))))

debt_to_income

[9.99,
 0.0191,
 0.4174,
 0.2028,
 0.0752,
 0.1846,
 0.0212,
 15.53,
 1.0,
 0.0422,
 0.695,
 0.4152,
 0.2173,
 0.2003,
 0.1155,
 0.4001,
 1.0,
 0.4803,
 0.0442,
 0.2578,
 1.0,
 1.0,
 0.0737,
 0.1029,
 25.22,
 0.0555,
 0.3186,
 0.3078,
 0.1037,
 0.6846,
 0.138,
 0.3045,
 0.3752,
 1.0,
 2.712,
 0.4016,
 0.2047,
 0.1853,
 0.1536,
 3.4639,
 1.0,
 1.0,
 29.45,
 398.5,
 1.0,
 1.0,
 0.1961,
 0.3194,
 1.0,
 19.06,
 0.2496,
 0.2223,
 0.0,
 0.2058,
 3.2,
 0.0852,
 0.1725,
 1.0,
 0.3826,
 0.0184,
 0.3305,
 0.3189,
 0.3245,
 0.0677,
 0.2198,
 10.331,
 0.453,
 0.0,
 0.2616,
 0.2383,
 0.352,
 0.499,
 0.0299,
 0.3381,
 0.1696,
 0.4544,
 0.099,
 0.3524,
 0.1461,
 1.0,
 3.3133,
 0.028,
 0.1671,
 0.1275,
 1.0,
 0.6704,
 0.3344,
 0.2197,
 0.0,
 -0.01,
 0.1948,
 0.4924,
 0.0831,
 0.3251,
 0.1862,
 0.1582,
 0.0349,
 0.0672,
 0.1484,
 0.1939,
 1.166,
 0.1931,
 0.0,
 -0.01,
 1.0,
 0.0136,
 1.0357,
 0.345,
 0.3874,
 0.3359,
 0.1789,
 0.3142,
 0.3457,
 0.0307,
 0.0985,
 0.0422,
 0.3117,
 0.4524,
 1.0,
 0.0085,

In [23]:
debt_to_income.index(83.8)

253

In [24]:
loans.debt_to_income_ratio = pd.DataFrame(debt_to_income)

In [26]:
loans.debt_to_income_ratio.head()

0    9.9900
1    0.0191
2    0.4174
3    0.2028
4    0.0752
Name: debt_to_income_ratio, dtype: float64

In [40]:
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
from category_encoders import OneHotEncoder
import numpy as np
imputer = preprocessing.Imputer(strategy="mean")
scaler = preprocessing.StandardScaler()

In [41]:
numerical_cols =  loans.drop(columns=[target]).select_dtypes(np.number).columns
ordinal_col = ['emp_length']
categorical_col = ['addr_state','title']

In [12]:
from sklearn.base import BaseEstimator

class ColumnSelector(BaseEstimator):
    def __init__(self, cols=None, drop_axis=False):
        self.cols = cols
        self.drop_axis = drop_axis

    def fit_transform(self, X, y=None):
        return self.transform(X=X, y=y)

    def transform(self, X, y=None):
        if hasattr(X, 'loc'):
            #only pandas dataframes have the method loc
            t = X.loc[:, self.cols].values
        else:
            # its a numpy array
            t = X[:, self.cols]

        if t.shape[-1] == 1 and self.drop_axis:
            t = t.reshape(-1)
        if len(t.shape) == 1 and not self.drop_axis:
            t = t[:, np.newaxis]
        return t

    def fit(self, X, y=None):
        return self

Numerical pipeline

In [56]:
numerical_pipeline = make_pipeline(
    ColumnSelector(cols=numerical_cols),
    imputer,
    scaler
)

numerical_pipeline.fit_transform(loans)[:5]

array([[ 0.40714321,  1.43390498],
       [-0.11757029, -0.5737501 ],
       [-0.09660996,  0.07388057],
       [-0.10790317, -0.24993476],
       [-0.11461806, -0.5737501 ]])

Categorical pipeline

In [31]:
categorical_pipeline = make_pipeline(
     ColumnSelector(cols=categorical_col),
     OneHotEncoder()
)

categorical_pipeline.fit_transform(loans)[:5]

,0_1,0_2,0_3,0_4,0_5,0_6,0_7,0_8,0_9,0_10,...,1_5,1_6,1_7,1_8,1_9,1_10,1_11,1_12,1_13,1_-1
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Ordinal pipeline

In [35]:
from category_encoders import OrdinalEncoder

# ColumnSelector's output is an array, so we use the column 0 for ordinal encoder
ordinal_encoder = OrdinalEncoder(mapping=[
    {"col": 0, 
      "mapping": {
        ("< 1 year", 0),
        ("1 year", 1),
        ("2 years",2),
        ("3 years", 3),
        ("4 years", 4),
        ("5 years", 5),
        ("6 years", 6),
        ("7 years", 7),
        ("8 years", 8),
        ("9 years", 9),
        ("10+ years", 10)
      } 
     }
])
ordinal_pipeline = make_pipeline(
    ColumnSelector(cols=ordinal_col),
    ordinal_encoder
)

ordinal_pipeline.fit_transform(loans)[:5]

,0
0,8
1,0
2,0
3,0
4,0


In [49]:
from sklearn.pipeline import make_union

In [80]:
processing_pipeline = make_union(
    numerical_pipeline,
    categorical_pipeline,
    ordinal_pipeline
)

In [51]:
processing_pipeline.fit_transform(loans)

array([[ 0.40714321,  1.43390498,  1.        , ...,  0.        ,
         0.        ,  8.        ],
       [-0.11757029, -0.5737501 ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.09660996,  0.07388057,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.11827546, -0.80042083,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.11857542, -0.83280236,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.06595093, -0.24993476,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

### Train a decision tree regressor with max features = 5 train it on the data and save the tree , what are its top 5 most important features?



In [57]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

tree = DecisionTreeClassifier(max_features=5)

In [58]:
independent_variables = loans.drop(columns=target).columns

In [65]:
tree_pipeline = make_pipeline(
    processing_pipeline,
    tree
)

In [61]:
tree_pipeline.fit(loans, loans[target])

Pipeline(memory=None,
     steps=[('featureunion', FeatureUnion(n_jobs=1,
       transformer_list=[('pipeline-1', Pipeline(memory=None,
     steps=[('columnselector', ColumnSelector(cols=Index(['debt_to_income_ratio', 'loan_amnt'], dtype='object'),
        drop_axis=False)), ('imputer', Imputer(axis=0, copy=True, missing_valu...it=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'))])

In [62]:
tree_pipeline.predict(loans)[:5]

array([1, 0, 0, 0, 0])

In [82]:
from sklearn.pipeline import FeatureUnion

In [85]:
loans_union = FeatureUnion(
    [('num',numerical_pipeline),
    ('cat',categorical_pipeline),
    ('ord',ordinal_pipeline)]
)

In [87]:
loans_union.fit_transform(loans, loans[target])

array([[ 0.40714321,  1.43390498,  1.        , ...,  0.        ,
         0.        ,  8.        ],
       [-0.11757029, -0.5737501 ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.09660996,  0.07388057,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.11827546, -0.80042083,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.11857542, -0.83280236,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.06595093, -0.24993476,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [88]:
tree_pipeline_2 = make_pipeline(
    loans_union,
    tree
)

In [94]:
tree_pipeline.steps[1][1].feature_importances_

array([5.10736557e-01, 2.90657355e-02, 3.49310691e-04, 2.59471292e-04,
       9.93887248e-08, 0.00000000e+00, 9.02394213e-08, 2.37946386e-03,
       2.19170576e-05, 7.16622054e-05, 0.00000000e+00, 1.38080005e-03,
       2.56239505e-04, 6.55826564e-06, 0.00000000e+00, 1.20177617e-03,
       4.52737408e-04, 0.00000000e+00, 2.40953146e-03, 4.10686970e-04,
       0.00000000e+00, 4.93017994e-04, 3.34485515e-05, 1.70713939e-05,
       6.97773280e-04, 1.74907924e-03, 6.64106659e-05, 2.67901017e-04,
       3.68180993e-04, 1.65738720e-04, 3.50756076e-04, 8.96039132e-04,
       8.24917369e-05, 4.57408374e-04, 1.41533099e-04, 0.00000000e+00,
       1.09975033e-04, 4.65314085e-05, 1.94527758e-05, 6.34139611e-07,
       1.55921623e-03, 3.17983269e-06, 0.00000000e+00, 1.16295190e-03,
       1.66370267e-03, 1.45040778e-03, 9.45389607e-06, 0.00000000e+00,
       0.00000000e+00, 9.04362142e-04, 0.00000000e+00, 5.98184570e-05,
       0.00000000e+00, 6.79739881e-03, 1.23835878e-03, 1.74348944e-03,
      

In [98]:
dict(zip(
    processing_pipeline.fit(loans).get_feature_names(),
    tree_pipeline.steps[1][1].feature_importances_
))

AttributeError: Transformer pipeline-1 (type Pipeline) does not provide get_feature_names.

In [76]:
tree_pipeline.steps[1]

('decisiontreeclassifier',
 DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
             max_features=5, max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             presort=False, random_state=None, splitter='best'))

In [70]:
import graphviz
from sklearn.tree import export_graphviz

def draw_tree(tree):
    dot_data = export_graphviz(tree, out_file=None, 
                         feature_names=loans.drop("status", axis=1).columns,  
                         filled=True, 
                         impurity=False,
                         rounded=True,  
                         special_characters=True)  
    
    graph = graphviz.Source(dot_data)
    graph.format = 'png'
    graph.render('tree',view=True)

In [79]:
export_graphviz(tree_pipeline.steps[1][1], out_file="tree.dot",
                    feature_names=independent_variables)

ValueError: Length of feature_names, 5 does not match number of features, 68

how do i get the feature names after the transformations????

### Create a predictive pipeline and perform hyperparameter optimization, using the r2 scoring. Which numerical transformer works best? which estimator works best, a KNN Classifier, or a DecisionTree Classifier?